[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter_garch_lecture_notebook.ipynb)

---

# Volatility Models: ARCH, GARCH and Extensions

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand volatility clustering and stylized facts of financial returns
2. Estimate and interpret ARCH and GARCH models
3. Apply asymmetric models (EGARCH, GJR-GARCH) to capture leverage effect
4. Perform model diagnostics and selection
5. Forecast volatility and calculate Value at Risk (VaR)

## Setup and Imports

In [ ]:
# Install arch if needed (for Colab)
try:
    from arch import arch_model
except ImportError:
    !pip install arch --quiet
    from arch import arch_model

# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Statistical models
from arch import arch_model
from arch.univariate import GARCH, EGARCH, ConstantMean
from statsmodels.stats.diagnostic import het_arch, acorr_ljungbox
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Plotting style
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['savefig.facecolor'] = 'none'
plt.rcParams['savefig.transparent'] = True
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

# Colors (IDA scheme)
COLORS = {
    'blue': '#1A3A6E',
    'red': '#DC3545',
    'green': '#2E7D32',
    'orange': '#E67E22',
    'gray': '#666666'
}

print("All libraries loaded successfully!")

## 1. Why Model Volatility?

**ARIMA models assume constant variance (homoskedasticity)**

Financial time series exhibit:
- **Volatility clustering**: Large changes followed by large changes
- **Fat tails (leptokurtosis)**: More extreme values than normal distribution
- **Leverage effect**: Negative returns increase volatility more than positive returns

These features require **conditional heteroskedasticity models**.

In [ ]:
# Simulate GARCH(1,1) process to demonstrate volatility clustering
np.random.seed(42)
n = 1000

# GARCH(1,1) parameters
omega = 0.00001
alpha = 0.10
beta = 0.85

# Initialize
sigma2 = np.zeros(n)
epsilon = np.zeros(n)
sigma2[0] = omega / (1 - alpha - beta)  # Unconditional variance

# Generate GARCH process
for t in range(1, n):
    sigma2[t] = omega + alpha * epsilon[t-1]**2 + beta * sigma2[t-1]
    epsilon[t] = np.sqrt(sigma2[t]) * np.random.randn()

returns = epsilon * 100  # Scale to percentage

print("GARCH(1,1) Simulation")
print(f"Parameters: omega={omega}, alpha={alpha}, beta={beta}")
print(f"Persistence: alpha + beta = {alpha + beta}")
print(f"Unconditional variance: {omega/(1-alpha-beta):.6f}")
print(f"Unconditional volatility: {np.sqrt(omega/(1-alpha-beta))*100:.2f}%")

In [ ]:
# Visualize volatility clustering
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Returns
axes[0].plot(returns, color=COLORS['blue'], linewidth=0.7, alpha=0.8)
axes[0].axhline(y=0, color='black', linewidth=0.5)
axes[0].set_ylabel('Returns (%)')
axes[0].set_title('Simulated GARCH(1,1) Returns: Volatility Clustering', fontweight='bold')

# Conditional volatility
axes[1].plot(np.sqrt(sigma2) * 100, color=COLORS['red'], linewidth=1)
axes[1].fill_between(range(n), 0, np.sqrt(sigma2) * 100, color=COLORS['red'], alpha=0.3)
axes[1].set_ylabel('Conditional Volatility (%)')
axes[1].set_xlabel('Time')
axes[1].set_title('True Conditional Volatility', fontweight='bold')

plt.tight_layout()
plt.show()

## 2. Stylized Facts of Financial Returns

Key empirical regularities:
1. **No autocorrelation** in returns $r_t$
2. **Significant autocorrelation** in $r_t^2$ and $|r_t|$
3. **Fat tails** (kurtosis > 3)
4. **Volatility clustering**

In [ ]:
# Check stylized facts
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# ACF of returns
plot_acf(returns, lags=30, ax=axes[0, 0], color=COLORS['blue'],
         vlines_kwargs={'color': COLORS['blue']}, alpha=0.05)
axes[0, 0].set_title('ACF of Returns (should be ~0)', fontweight='bold')

# ACF of squared returns
plot_acf(returns**2, lags=30, ax=axes[0, 1], color=COLORS['red'],
         vlines_kwargs={'color': COLORS['red']}, alpha=0.05)
axes[0, 1].set_title('ACF of Squared Returns (significant!)', fontweight='bold')

# Distribution vs Normal
axes[1, 0].hist(returns, bins=50, density=True, color=COLORS['blue'],
                alpha=0.7, edgecolor='white', label='Data')
x = np.linspace(returns.min(), returns.max(), 100)
axes[1, 0].plot(x, stats.norm.pdf(x, returns.mean(), returns.std()),
                color=COLORS['red'], linewidth=2, label='Normal')
axes[1, 0].set_title(f'Distribution: Kurtosis = {stats.kurtosis(returns)+3:.2f} (Normal = 3)', fontweight='bold')
axes[1, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=2)

# QQ-Plot
stats.probplot(returns, dist="norm", plot=axes[1, 1])
axes[1, 1].get_lines()[0].set_color(COLORS['blue'])
axes[1, 1].get_lines()[0].set_markersize(4)
axes[1, 1].get_lines()[1].set_color(COLORS['red'])
axes[1, 1].set_title('QQ-Plot: Fat Tails Visible', fontweight='bold')

plt.tight_layout()
plt.show()

print("\nStylized Facts Summary:")
print(f"  Mean return: {returns.mean():.4f}%")
print(f"  Std deviation: {returns.std():.4f}%")
print(f"  Skewness: {stats.skew(returns):.4f}")
print(f"  Kurtosis: {stats.kurtosis(returns)+3:.4f} (Normal = 3)")

## 3. Testing for ARCH Effects

**Engle's ARCH-LM Test:**
1. Estimate mean model, get residuals $\hat{\varepsilon}_t$
2. Regress $\hat{\varepsilon}_t^2$ on its lags
3. Test statistic: $LM = T \cdot R^2 \sim \chi^2(q)$

- $H_0$: No ARCH effects
- $H_1$: ARCH effects present

In [ ]:
# ARCH-LM test
print("ARCH-LM Test for Heteroskedasticity")
print("="*50)

# Test with different lag orders
for q in [5, 10, 20]:
    lm_stat, lm_pvalue, f_stat, f_pvalue = het_arch(returns - returns.mean(), nlags=q)
    result = "Reject H0 → ARCH present" if lm_pvalue < 0.05 else "Cannot reject H0"
    print(f"  Lags = {q}: LM = {lm_stat:.2f}, p-value = {lm_pvalue:.4f} → {result}")

print("\n⚠️ ARCH effects detected! We need GARCH modeling.")

## 4. The ARCH(q) Model

**Engle (1982) - Nobel Prize 2003**

$$\varepsilon_t = \sigma_t z_t, \quad z_t \sim \text{i.i.d.}(0, 1)$$
$$\sigma_t^2 = \omega + \alpha_1 \varepsilon_{t-1}^2 + \cdots + \alpha_q \varepsilon_{t-q}^2$$

**Constraints:**
- $\omega > 0$
- $\alpha_i \geq 0$
- $\sum \alpha_i < 1$ (stationarity)

In [ ]:
# Estimate ARCH(5) model
print("ARCH(5) Model Estimation")
print("="*50)

model_arch = arch_model(returns, vol='ARCH', p=5, dist='normal')
res_arch = model_arch.fit(disp='off')

print(res_arch.summary())

## 5. The GARCH(p,q) Model

**Bollerslev (1986)** - Adds lagged conditional variance for persistence:

$$\sigma_t^2 = \omega + \sum_{i=1}^{q} \alpha_i \varepsilon_{t-i}^2 + \sum_{j=1}^{p} \beta_j \sigma_{t-j}^2$$

**GARCH(1,1) - The workhorse model:**
$$\sigma_t^2 = \omega + \alpha \varepsilon_{t-1}^2 + \beta \sigma_{t-1}^2$$

- $\alpha$ = reaction to news (ARCH effect)
- $\beta$ = persistence (GARCH effect)
- $\alpha + \beta$ = total persistence

In [ ]:
# Estimate GARCH(1,1) model
print("GARCH(1,1) Model Estimation")
print("="*50)

model_garch = arch_model(returns, vol='Garch', p=1, q=1, dist='normal')
res_garch = model_garch.fit(disp='off')

print(res_garch.summary())

In [ ]:
# Interpret GARCH(1,1) parameters
omega_est = res_garch.params['omega']
alpha_est = res_garch.params['alpha[1]']
beta_est = res_garch.params['beta[1]']

print("GARCH(1,1) Parameter Interpretation")
print("="*50)
print(f"\nEstimated Parameters:")
print(f"  ω (omega) = {omega_est:.6f}")
print(f"  α (alpha) = {alpha_est:.4f}  ← News reaction")
print(f"  β (beta)  = {beta_est:.4f}  ← Persistence")

print(f"\nDerived Quantities:")
persistence = alpha_est + beta_est
print(f"  α + β = {persistence:.4f}  ← Total persistence")

if persistence < 1:
    uncond_var = omega_est / (1 - persistence)
    uncond_vol = np.sqrt(uncond_var)
    half_life = np.log(0.5) / np.log(persistence)
    print(f"  Unconditional variance: {uncond_var:.6f}")
    print(f"  Unconditional volatility: {uncond_vol*100:.2f}%")
    print(f"  Half-life: {half_life:.1f} periods")
else:
    print("  ⚠️ Model is IGARCH (α + β = 1), unconditional variance undefined")

In [ ]:
# Compare true vs estimated volatility
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(np.sqrt(sigma2) * 100, color=COLORS['blue'], linewidth=1, alpha=0.7, label='True Volatility')
ax.plot(res_garch.conditional_volatility, color=COLORS['red'], linewidth=1, label='GARCH(1,1) Estimated')
ax.set_ylabel('Volatility (%)')
ax.set_xlabel('Time')
ax.set_title('True vs Estimated Conditional Volatility', fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2)

plt.tight_layout()
plt.show()

# Correlation
corr = np.corrcoef(np.sqrt(sigma2), res_garch.conditional_volatility)[0, 1]
print(f"\nCorrelation between true and estimated volatility: {corr:.4f}")

## 6. Alternative Distributions

Normal distribution underestimates tail risk. Alternatives:
- **Student-t**: Fat tails, estimated degrees of freedom $\nu$
- **GED**: Generalized Error Distribution
- **Skewed Student-t**: Asymmetry + fat tails

In [ ]:
# Compare different distributions
print("GARCH(1,1) with Different Distributions")
print("="*60)

distributions = ['normal', 't', 'skewt', 'ged']
results = {}

for dist in distributions:
    try:
        model = arch_model(returns, vol='Garch', p=1, q=1, dist=dist)
        res = model.fit(disp='off')
        results[dist] = res
        print(f"{dist:>8}: AIC = {res.aic:.2f}, BIC = {res.bic:.2f}, LogLik = {res.loglikelihood:.2f}")
    except:
        print(f"{dist:>8}: Failed to converge")

# Best model
best_dist = min(results, key=lambda x: results[x].aic)
print(f"\nBest distribution by AIC: {best_dist}")

In [ ]:
# Student-t GARCH details
if 't' in results:
    print("GARCH(1,1) with Student-t Distribution")
    print("="*50)
    print(results['t'].summary())

## 7. Asymmetric GARCH Models

### Leverage Effect
Negative returns increase volatility MORE than positive returns of the same magnitude.

Standard GARCH uses $\varepsilon_{t-1}^2$ → symmetric response

### Solutions:
- **EGARCH** (Nelson, 1991)
- **GJR-GARCH** (Glosten, Jagannathan, Runkle, 1993)
- **TGARCH** (Zakoian, 1994)

In [ ]:
# Simulate data with leverage effect
np.random.seed(456)
n = 1000

# GJR-GARCH parameters with leverage
omega = 0.0001
alpha = 0.05
gamma = 0.12  # Leverage effect
beta = 0.85

sigma2_lev = np.zeros(n)
epsilon_lev = np.zeros(n)
sigma2_lev[0] = omega / (1 - alpha - gamma/2 - beta)

for t in range(1, n):
    indicator = 1 if epsilon_lev[t-1] < 0 else 0
    sigma2_lev[t] = omega + alpha * epsilon_lev[t-1]**2 + gamma * epsilon_lev[t-1]**2 * indicator + beta * sigma2_lev[t-1]
    epsilon_lev[t] = np.sqrt(sigma2_lev[t]) * np.random.randn()

returns_lev = epsilon_lev * 100

print("Simulated GJR-GARCH with Leverage Effect")
print(f"  α = {alpha}, γ = {gamma}, β = {beta}")
print(f"  Impact of positive shock: α = {alpha}")
print(f"  Impact of negative shock: α + γ = {alpha + gamma}")
print(f"  Ratio: {(alpha + gamma)/alpha:.2f}x higher for negative shocks!")

In [ ]:
# Visualize leverage effect
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter: returns vs next-period volatility
next_vol = np.sqrt(sigma2_lev[1:]) * 100
prev_ret = returns_lev[:-1]

colors_scatter = [COLORS['red'] if r < 0 else COLORS['blue'] for r in prev_ret]
axes[0].scatter(prev_ret, next_vol, c=colors_scatter, alpha=0.4, s=15)
axes[0].axvline(x=0, color='gray', linestyle='--', linewidth=1)
axes[0].set_xlabel('Return at t (%)')
axes[0].set_ylabel('Volatility at t+1 (%)')
axes[0].set_title('Leverage Effect: Negative Shocks → Higher Volatility', fontweight='bold')

# Box plot
neg_mask = prev_ret < 0
bp = axes[1].boxplot([next_vol[neg_mask], next_vol[~neg_mask]],
                      labels=['After Negative Shock', 'After Positive Shock'],
                      patch_artist=True)
bp['boxes'][0].set_facecolor(COLORS['red'])
bp['boxes'][0].set_alpha(0.6)
bp['boxes'][1].set_facecolor(COLORS['blue'])
bp['boxes'][1].set_alpha(0.6)
axes[1].set_ylabel('Next Period Volatility (%)')
axes[1].set_title('Distribution of Volatility by Shock Sign', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nMean volatility after negative shock: {next_vol[neg_mask].mean():.3f}%")
print(f"Mean volatility after positive shock: {next_vol[~neg_mask].mean():.3f}%")

In [ ]:
# Estimate asymmetric models
print("Asymmetric GARCH Models Comparison")
print("="*60)

# Standard GARCH
model_garch_lev = arch_model(returns_lev, vol='Garch', p=1, q=1, dist='t')
res_garch_lev = model_garch_lev.fit(disp='off')

# GJR-GARCH (o=1 adds asymmetry)
model_gjr = arch_model(returns_lev, vol='Garch', p=1, o=1, q=1, dist='t')
res_gjr = model_gjr.fit(disp='off')

# EGARCH
model_egarch = arch_model(returns_lev, vol='EGARCH', p=1, q=1, dist='t')
res_egarch = model_egarch.fit(disp='off')

print(f"{'Model':<15} {'AIC':>10} {'BIC':>10} {'LogLik':>12}")
print("-"*50)
print(f"{'GARCH(1,1)':<15} {res_garch_lev.aic:>10.2f} {res_garch_lev.bic:>10.2f} {res_garch_lev.loglikelihood:>12.2f}")
print(f"{'GJR-GARCH':<15} {res_gjr.aic:>10.2f} {res_gjr.bic:>10.2f} {res_gjr.loglikelihood:>12.2f}")
print(f"{'EGARCH':<15} {res_egarch.aic:>10.2f} {res_egarch.bic:>10.2f} {res_egarch.loglikelihood:>12.2f}")

print("\n→ Lower AIC/BIC = Better model")

In [ ]:
# GJR-GARCH details
print("GJR-GARCH(1,1,1) Estimation Results")
print("="*50)
print(res_gjr.summary())

In [ ]:
# EGARCH details
print("EGARCH(1,1) Estimation Results")
print("="*50)
print(res_egarch.summary())

print("\nInterpretation of EGARCH parameters:")
print(f"  gamma[1] = {res_egarch.params['gamma[1]']:.4f}")
if res_egarch.params['gamma[1]'] < 0:
    print("  → Negative gamma confirms leverage effect!")
    print("  → Negative shocks increase volatility more than positive shocks")

## 8. News Impact Curve

The **News Impact Curve** shows how today's volatility $\sigma_{t+1}^2$ responds to yesterday's shock $\varepsilon_t$, holding $\sigma_t^2$ constant.

In [ ]:
# Plot News Impact Curves
epsilon_range = np.linspace(-0.04, 0.04, 200)
sigma2_prev = 0.0004  # Fixed previous variance

# GARCH (symmetric)
omega_g = 0.0001
alpha_g = 0.10
beta_g = 0.85
sigma2_garch_curve = omega_g + alpha_g * epsilon_range**2 + beta_g * sigma2_prev

# GJR-GARCH
omega_gjr = 0.0001
alpha_gjr = 0.05
gamma_gjr = 0.10
beta_gjr = 0.85
indicator = (epsilon_range < 0).astype(float)
sigma2_gjr_curve = omega_gjr + alpha_gjr * epsilon_range**2 + gamma_gjr * epsilon_range**2 * indicator + beta_gjr * sigma2_prev

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(epsilon_range * 100, np.sqrt(sigma2_garch_curve) * 100,
        color=COLORS['blue'], linewidth=2, label='GARCH (Symmetric)')
ax.plot(epsilon_range * 100, np.sqrt(sigma2_gjr_curve) * 100,
        color=COLORS['red'], linewidth=2, label='GJR-GARCH (Asymmetric)')

ax.axvline(x=0, color='gray', linestyle=':', linewidth=1)
ax.set_xlabel('Shock εₜ (%)', fontsize=12)
ax.set_ylabel('Conditional Volatility σₜ₊₁ (%)', fontsize=12)
ax.set_title('News Impact Curve: GARCH vs GJR-GARCH', fontweight='bold', fontsize=14)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2)

plt.tight_layout()
plt.show()

print("The asymmetric curve shows higher volatility for negative shocks.")

## 9. Model Diagnostics

After fitting a GARCH model, check:
1. **Standardized residuals** $\hat{z}_t = \hat{\varepsilon}_t / \hat{\sigma}_t$ should be i.i.d.
2. **No remaining ARCH effects** in $\hat{z}_t^2$
3. **No autocorrelation** in $\hat{z}_t$

In [ ]:
# Diagnostics for GJR-GARCH
std_resid = res_gjr.std_resid

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Time series
axes[0, 0].plot(std_resid, color=COLORS['blue'], linewidth=0.7, alpha=0.8)
axes[0, 0].axhline(y=0, color='black', linewidth=0.5)
axes[0, 0].axhline(y=2, color=COLORS['red'], linestyle='--', linewidth=0.8, alpha=0.5)
axes[0, 0].axhline(y=-2, color=COLORS['red'], linestyle='--', linewidth=0.8, alpha=0.5)
axes[0, 0].set_title('Standardized Residuals', fontweight='bold')
axes[0, 0].set_ylabel('zₜ')

# ACF of squared standardized residuals
plot_acf(std_resid**2, lags=20, ax=axes[0, 1], color=COLORS['blue'],
         vlines_kwargs={'color': COLORS['blue']}, alpha=0.05)
axes[0, 1].set_title('ACF of z²ₜ (should be ≈ 0)', fontweight='bold')

# Histogram
axes[1, 0].hist(std_resid, bins=40, density=True, color=COLORS['blue'],
                alpha=0.7, edgecolor='white')
x = np.linspace(-5, 5, 100)
axes[1, 0].plot(x, stats.norm.pdf(x), color=COLORS['red'], linewidth=2, label='N(0,1)')
axes[1, 0].plot(x, stats.t.pdf(x, df=6), color=COLORS['green'], linewidth=2, label='t(6)')
axes[1, 0].set_title('Distribution of Standardized Residuals', fontweight='bold')
axes[1, 0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2)

# QQ-plot
stats.probplot(std_resid, dist="norm", plot=axes[1, 1])
axes[1, 1].get_lines()[0].set_color(COLORS['blue'])
axes[1, 1].get_lines()[0].set_markersize(4)
axes[1, 1].get_lines()[1].set_color(COLORS['red'])
axes[1, 1].set_title('QQ-Plot of Standardized Residuals', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Formal diagnostic tests
print("Diagnostic Tests for GJR-GARCH Model")
print("="*50)

# Ljung-Box on standardized residuals
lb_z = acorr_ljungbox(std_resid, lags=10, return_df=True)
print("\n1. Ljung-Box Test on zₜ (no autocorrelation in mean):")
print(f"   Lag 10: Q = {lb_z['lb_stat'].iloc[-1]:.2f}, p-value = {lb_z['lb_pvalue'].iloc[-1]:.4f}")
print(f"   Result: {'✓ No autocorrelation' if lb_z['lb_pvalue'].iloc[-1] > 0.05 else '✗ Autocorrelation present'}")

# Ljung-Box on squared standardized residuals
lb_z2 = acorr_ljungbox(std_resid**2, lags=10, return_df=True)
print("\n2. Ljung-Box Test on z²ₜ (no remaining ARCH effects):")
print(f"   Lag 10: Q = {lb_z2['lb_stat'].iloc[-1]:.2f}, p-value = {lb_z2['lb_pvalue'].iloc[-1]:.4f}")
print(f"   Result: {'✓ No ARCH effects' if lb_z2['lb_pvalue'].iloc[-1] > 0.05 else '✗ ARCH effects remain'}")

# ARCH-LM test on standardized residuals
lm_stat, lm_pval, _, _ = het_arch(std_resid, nlags=5)
print("\n3. ARCH-LM Test on Standardized Residuals:")
print(f"   LM = {lm_stat:.2f}, p-value = {lm_pval:.4f}")
print(f"   Result: {'✓ No remaining ARCH' if lm_pval > 0.05 else '✗ ARCH effects remain'}")

## 10. Volatility Forecasting

**One-step forecast:**
$$\hat{\sigma}_{T+1}^2 = \omega + \alpha \varepsilon_T^2 + \beta \sigma_T^2$$

**Multi-step forecast:** Converges to unconditional variance
$$E_T[\sigma_{T+h}^2] = \bar{\sigma}^2 + (\alpha + \beta)^{h-1} (\sigma_{T+1}^2 - \bar{\sigma}^2)$$

In [ ]:
# Volatility forecasting
horizon = 50

# Generate forecasts
forecasts = res_gjr.forecast(horizon=horizon)
vol_forecast = np.sqrt(forecasts.variance.values[-1, :])

# Get historical volatility
hist_vol = res_gjr.conditional_volatility

# Unconditional volatility
params = res_gjr.params
omega_hat = params['omega']
alpha_hat = params['alpha[1]']
gamma_hat = params['gamma[1]']
beta_hat = params['beta[1]']
persistence = alpha_hat + gamma_hat/2 + beta_hat
uncond_vol = np.sqrt(omega_hat / (1 - persistence)) if persistence < 1 else np.nan

# Plot
fig, ax = plt.subplots(figsize=(14, 5))

# Historical (last 100 periods)
ax.plot(range(len(hist_vol)-100, len(hist_vol)), hist_vol[-100:],
        color=COLORS['blue'], linewidth=1, label='Historical Volatility')

# Forecast
forecast_start = len(hist_vol)
ax.plot(range(forecast_start-1, forecast_start + horizon),
        np.concatenate([[hist_vol[-1]], vol_forecast]),
        color=COLORS['red'], linewidth=2, linestyle='--', label='Forecast')

# Unconditional level
if not np.isnan(uncond_vol):
    ax.axhline(y=uncond_vol, color=COLORS['green'], linestyle=':',
               linewidth=1.5, label=f'Unconditional: {uncond_vol:.2f}%')

ax.axvline(x=forecast_start-1, color='black', linestyle='-', alpha=0.3)
ax.set_xlabel('Time')
ax.set_ylabel('Volatility (%)')
ax.set_title('GJR-GARCH Volatility Forecast', fontweight='bold')
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=3)

plt.tight_layout()
plt.show()

print(f"\n1-step ahead forecast: {vol_forecast[0]:.2f}%")
print(f"10-step ahead forecast: {vol_forecast[9]:.2f}%")
print(f"Unconditional volatility: {uncond_vol:.2f}%")

## 11. Value at Risk (VaR)

**VaR at level $\alpha$**: Maximum loss that will not be exceeded with probability $1-\alpha$

$$\text{VaR}_\alpha = -\mu_{t+1} + z_\alpha \cdot \sigma_{t+1}$$

For normal: $z_{0.05} = 1.645$, $z_{0.01} = 2.326$

For Student-t: Use t-quantiles (fatter tails = higher VaR)

In [ ]:
# Calculate VaR
print("Value at Risk Calculation")
print("="*50)

portfolio_value = 1_000_000  # EUR
sigma_1 = vol_forecast[0] / 100  # 1-step ahead volatility (as decimal)

# Normal distribution
z_95 = stats.norm.ppf(0.95)
z_99 = stats.norm.ppf(0.99)

VaR_95_normal = z_95 * sigma_1 * portfolio_value
VaR_99_normal = z_99 * sigma_1 * portfolio_value

print(f"\nPortfolio value: €{portfolio_value:,.0f}")
print(f"1-day volatility forecast: {sigma_1*100:.2f}%")

print(f"\nNormal Distribution:")
print(f"  VaR 95% (1 day): €{VaR_95_normal:,.0f}")
print(f"  VaR 99% (1 day): €{VaR_99_normal:,.0f}")

# Student-t distribution (estimated df from model)
if 'nu' in res_gjr.params.index:
    nu = res_gjr.params['nu']
    t_95 = stats.t.ppf(0.95, df=nu) * np.sqrt((nu-2)/nu)  # Adjust for unit variance
    t_99 = stats.t.ppf(0.99, df=nu) * np.sqrt((nu-2)/nu)
    
    VaR_95_t = t_95 * sigma_1 * portfolio_value
    VaR_99_t = t_99 * sigma_1 * portfolio_value
    
    print(f"\nStudent-t Distribution (df={nu:.1f}):")
    print(f"  VaR 95% (1 day): €{VaR_95_t:,.0f}")
    print(f"  VaR 99% (1 day): €{VaR_99_t:,.0f}")

# 10-day VaR (scaling rule)
print(f"\n10-day VaR (scaling by √10):")
print(f"  VaR 99% (10 days): €{VaR_99_normal * np.sqrt(10):,.0f}")

## Summary

### Key Takeaways

1. **ARIMA assumes constant variance** - not realistic for financial data

2. **GARCH(1,1)** is the workhorse model:
   - $\sigma_t^2 = \omega + \alpha \varepsilon_{t-1}^2 + \beta \sigma_{t-1}^2$
   - α = news reaction, β = persistence
   - Stationarity: α + β < 1

3. **Leverage effect** requires asymmetric models:
   - EGARCH: log-specification, no positivity constraints
   - GJR-GARCH: indicator function for negative shocks

4. **Student-t distribution** captures fat tails better than normal

5. **Applications:**
   - VaR and risk management
   - Option pricing
   - Portfolio optimization

### Practical Workflow
1. Test for ARCH effects (ARCH-LM test)
2. Estimate GARCH(1,1) with Student-t
3. Check for asymmetry (GJR/EGARCH)
4. Diagnostic checks on standardized residuals
5. Forecast volatility, calculate VaR